In [1]:
import numpy as np
import pandas as pd

from sklearn.impute import (SimpleImputer,KNNImputer)
from sklearn.ensemble import (RandomForestRegressor, IsolationForest)
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import lightgbm as lgb
from sklearn.ensemble import ExtraTreesRegressor

from sklearn.metrics import r2_score

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def fill_missing_values(X, n_neighbors = 75, method="KNN"): 
    
    # normalization
    X_std = np.nanstd(X,axis=0,keepdims=True)
    X_ave = np.nanmean(X,axis=0,keepdims=True)
    X_norma = (X-X_ave)/X_std
    
    # use KNNImputer
    imputer = KNNImputer(missing_values=np.nan, n_neighbors=n_neighbors, weights = 'distance') if method=="KNN"\
        else SimpleImputer(missing_values=np.nan, strategy='median')
    
    X_norma_fixed = imputer.fit_transform(X_norma)
    
    return X_norma_fixed

def remove_outliers(X, y):
    print("IsolationForest-Traing data shape before removed: {}".format(X.shape))
    Z = np.c_[X, y]
    iforest = IsolationForest(max_samples=200, random_state=1, contamination=0.005)
    iforest.fit(Z)
    iforest_outlier_pred = iforest.predict(Z)
    
    Z = np.c_[X, y]
    local = LocalOutlierFactor(n_neighbors=150, contamination=0.005)
    local.fit(Z)
    local_outlier_pred = local.fit_predict(Z)

    mask = np.logical_and((iforest_outlier_pred!=-1), (local_outlier_pred!=-1))
    X , y = X[mask, :], y[mask]
    print("IsolationForest-Traing data shape after removed: {}".format(X.shape))
    return X, y

def select_features(X, y, X_test, feature_num=50):
    rf = RandomForestRegressor(n_jobs=-1, n_estimators=80, random_state=1)
    rf.fit(X, y)
    indices = np.asarray(list(rf.feature_importances_)).argsort()[-feature_num:][::-1]
    
    X = np.take(X, indices, axis = 1)
    X_test = np.take(X_test, indices, axis = 1)
    return X, X_test

In [3]:

X_train_data = pd.read_csv('X_train.csv')
y_train_data = pd.read_csv('y_train.csv')
X_test_data = pd.read_csv('X_test.csv')

indices_test = np.array(X_test_data)[:,0]
X_test = np.array(X_test_data)[:,1:]
y_train = np.array(y_train_data)[:,1]
X_train = np.array(X_train_data)[:,1:]

## 1. Imputation of Missing Values
* [Reference](https://scikit-learn.org/stable/modules/impute.html)
* We use median of column instead of mean

In [4]:
X_train_missing_indices = X_train[X_train==np]
X_train = fill_missing_values(X_train, n_neighbors=75)
X_test_processed = fill_missing_values(X_test)
print(X_train.shape)
print(X_test_processed.shape)

/var/folders/df/qjssdfqd3f52ltwv16hkg4kh0000gn/T/ipykernel_70368/2723553188.py:6: RuntimeWarning: invalid value encountered in true_divide
  X_norma = (X-X_ave)/X_std
/var/folders/df/qjssdfqd3f52ltwv16hkg4kh0000gn/T/ipykernel_70368/2723553188.py:6: RuntimeWarning: invalid value encountered in true_divide
  X_norma = (X-X_ave)/X_std


(1212, 828)
(776, 828)


## 2. Outlier Detection
* [reference_sklearn](https://scikit-learn.org/stable/modules/outlier_detection.html)
* [reference_in_detail](https://practicaldatascience.co.uk/machine-learning/how-to-use-the-isolation-forest-model-for-outlier-detection)

In [5]:
X_processed,y_processed = remove_outliers(X_train,y_train)

IsolationForest-Traing data shape before removed: (1212, 828)
IsolationForest-Traing data shape after removed: (1200, 828)


## 3. Feature Selection

In [6]:
# pearsonr feature selection
from scipy.stats import pearsonr
from sklearn.decomposition import PCA

def pearsonr_feature_selection(X, y, X_test,degree,weights):
    feature_idx =  np.array([i for i in range(0,X.shape[1])])
    correlations = {}
#     print("feature_idx: ", feature_idx.T)
    for idx in feature_idx:
        correlation = 0
        for degree_idx in range(0,len(weights)):
            correlation += abs(weights[degree_idx]*pearsonr(X[:,idx],y**degree[degree_idx])[0])
        correlations[idx] = correlation
    data_correlations = pd.DataFrame(correlations, index=['correlation']).T
    indices_desc = data_correlations['correlation'].abs().sort_values(ascending=False).index
#     print(indices_desc)
    return data_correlations,indices_desc

def pearsonr_feature_selection_correlation(X, X_test,threshold):
    feature_idx =  np.array([i for i in range(0,X.shape[1])])
    correlation_select = np.array([True for i in range(0,X.shape[1])])
    for f_idx in feature_idx:
        if(correlation_select[f_idx]):
            feature_co,indices_desc = pearsonr_feature_selection(X[:,(f_idx+1):], X[:,f_idx], X_test_processed,np.array([1]),np.array([1]))
            reomove_features = feature_co.loc[indices_desc][abs(feature_co['correlation']) >= threshold]
            remove_index = reomove_features.index+(f_idx+1)
            correlation_select[remove_index] = False
    
    return X[:,correlation_select],X_test[:,correlation_select]

In [7]:
def LGB_feature_selection(X_processed,X_test_processed,y_processed):
    data_relation,indices_desc = pearsonr_feature_selection(X_processed, y_processed, X_test_processed,np.array([1,-3,3]),np.array([0.9,0.05,0.05]))
    selected_features = data_relation.loc[indices_desc][abs(data_relation['correlation']) >= 0.025]
    X_selection = X_processed[:,selected_features.index]
    X_test_selection = X_test_processed[:,selected_features.index]
    X_selection_tree, X_test_selection_tree = select_features(X_selection, y_processed, X_test_selection,feature_num = 105)
    X_selection_tree_cor, X_test_selection_tree_cor = pearsonr_feature_selection_correlation(X_selection_tree, X_test_selection_tree,0.93)
    return X_selection_tree_cor,X_test_selection_tree_cor

In [8]:
def GP_feature_selection(X_processed,X_test_processed,y_processed):
    data_relation,indices_desc = pearsonr_feature_selection(X_processed, y_processed, X_test_processed,np.array([1,-3,3]),np.array([0.95,0.025,0.025]))
    selected_features = data_relation.loc[indices_desc][abs(data_relation['correlation']) >= 0.001]
    X_selection = X_processed[:,selected_features.index]
    X_test_selection = X_test_processed[:,selected_features.index]
    X_selection_tree, X_test_selection_tree = select_features(X_selection, y_processed, X_test_selection,feature_num = 54)
    X_selection_tree_cor, X_test_selection_tree_cor = pearsonr_feature_selection_correlation(X_selection_tree, X_test_selection_tree,0.96)
    return X_selection_tree_cor,X_test_selection_tree_cor

In [9]:
X_lgb,X_test_lgb = LGB_feature_selection(X_processed,X_test_processed,y_processed)

/var/folders/df/qjssdfqd3f52ltwv16hkg4kh0000gn/T/ipykernel_70368/1493064456.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  selected_features = data_relation.loc[indices_desc][abs(data_relation['correlation']) >= 0.025]
/var/folders/df/qjssdfqd3f52ltwv16hkg4kh0000gn/T/ipykernel_70368/4248551713.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  reomove_features = feature_co.loc[indices_desc][abs(feature_co['correlation']) >= threshold]


In [10]:
X_gp,X_test_gp = GP_feature_selection(X_processed,X_test_processed,y_processed)

/var/folders/df/qjssdfqd3f52ltwv16hkg4kh0000gn/T/ipykernel_70368/3447698863.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  selected_features = data_relation.loc[indices_desc][abs(data_relation['correlation']) >= 0.001]
/var/folders/df/qjssdfqd3f52ltwv16hkg4kh0000gn/T/ipykernel_70368/4248551713.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  reomove_features = feature_co.loc[indices_desc][abs(feature_co['correlation']) >= threshold]


In [11]:
X_extra,X_test_extra = select_features(X_processed, y_processed, X_test_processed,feature_num = 50)

## 4. Gaussian Process and lgb

In [12]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, DotProduct, WhiteKernel, RationalQuadratic

def custom_r2(prediction, train_data):
    """Regular r2 cost function returned as a tuple to be used with lgb"""
    labels = train_data.get_label()
    return 'r2', r2_score(labels, prediction), True

def fit_model_and_pred_extra(X_train, y_train, X_val, y_val, X_test):
    model = ExtraTreesRegressor(n_jobs=1, max_depth=None, n_estimators=190, random_state=0, min_samples_split=3, max_features=None)

    model.fit(X_train, y_train)
    
    y_val_pred = model.predict(X_val)
    score = r2_score(y_val, y_val_pred)
    
    y_pred = model.predict(X_test) 
    
    return y_val_pred, y_pred

def fit_model_and_pred_gp(X_train, y_train, X_val, y_val, X_test):
    kernel = RationalQuadratic(length_scale=0.5, alpha=1.0, length_scale_bounds=(1e-4, 10))
    gpr = GaussianProcessRegressor(kernel=kernel, random_state=0, n_restarts_optimizer=1, alpha=0.07, normalize_y=True)
    gpr.fit(X_train, y_train)
    y_val_pred = gpr.predict(X_val)   
    y_pred = gpr.predict(X_test) 

    return y_val_pred, y_pred

def fit_model_and_pred_lgb(X_train, y_train, X_val, y_val, X_test):
   
    params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'num_leaves': 1800,
        'learning_rate': 0.025,
        'max_depth': 11,
        'n_estimators': 1000,
        'min_child_weight': 1,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'feature_fraction': 0.5,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': -1,
        'num_iterations':1000,
    }  
    
    
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=500,
                    feval=custom_r2,
                    valid_sets={lgb_train, lgb_eval},
                    early_stopping_rounds=100,
                    verbose_eval=False
                   )
    
    y_val_pred = gbm.predict(X_val)
    score = r2_score(y_val, y_val_pred)
    y_pred = gbm.predict(X_test) 
    return y_val_pred, y_pred

def train_k_fold_combine(X_lgb,X_test_lgb,X_gp,X_test_gp, X_extra,X_test_extra, y ,w1=0.33, w2=0.33, w3=0.33, fold_num=10):
    kf = KFold(n_splits=fold_num, shuffle=False)
    kf.get_n_splits(X_lgb)
    test_score = 0.0
    train_score = 0.0
    i = 0
    for train_index, test_index in kf.split(X_lgb):
        X_lgb_train, X_lgb_val = X_lgb[train_index], X_lgb[test_index]
        X_gp_train, X_gp_val = X_gp[train_index], X_gp[test_index]
        X_extra_train, X_extra_val = X_extra[train_index], X_extra[test_index]
        y_train, y_val = y[train_index], y[test_index]

        _, y_pred1 = fit_model_and_pred_gp(X_gp_train, y_train, X_gp_val, y_val, X_gp_val)
        _, y_pred2 = fit_model_and_pred_lgb(X_lgb_train, y_train, X_lgb_val, y_val, X_lgb_val)
        _, y_pred3 = fit_model_and_pred_extra(X_extra_train, y_train, X_extra_val, y_val, X_extra_val)
        y_pred = w1*y_pred1 + w2*y_pred2 + w3 * y_pred3

        
        score = r2_score(y_val, y_pred)
        print(i+1, 'th . the obtained validation r2 score is : ',score)
        test_score += score
        print(' ')
        i += 1
       
    print("Validation score: %f"%(test_score/fold_num))
    
def train_k_fold_combine_predict(X_lgb,X_test_lgb,X_gp,X_test_gp, X_extra,X_test_extra, y ,w1=0.33, w2=0.33, w3=0.33, fold_num=10):
    kf = KFold(n_splits=fold_num, shuffle=False)
    kf.get_n_splits(X_lgb)
    test_score = 0.0
    train_score = 0.0
    cnt = 0
    y_test_predict = np.zeros(X_test.shape[0])

    for train_index, test_index in kf.split(X_lgb):
        X_lgb_train, X_lgb_val = X_lgb[train_index], X_lgb[test_index]
        X_gp_train, X_gp_val = X_gp[train_index], X_gp[test_index]
        X_extra_train, X_extra_val = X_extra[train_index], X_extra[test_index]
        y_train, y_val = y[train_index], y[test_index]

        y_val_pred1, y_pred1 = fit_model_and_pred_gp(X_gp_train, y_train, X_gp_val, y_val, X_test_gp)
        y_val_pred2, y_pred2 = fit_model_and_pred_lgb(X_lgb_train, y_train, X_lgb_val, y_val, X_test_lgb)
        y_val_pred3, y_pred3 = fit_model_and_pred_extra(X_extra_train, y_train, X_extra_val, y_val, X_test_extra)
        
        y_pred = w1*y_pred1 + w2*y_pred2 + w3 * y_pred3
        y_val_pred = w1*y_val_pred1 + w2*y_val_pred2 + w3 * y_val_pred3
        score = r2_score(y_val, y_val_pred)

        y_test_predict += y_pred
       
    return y_test_predict/fold_num


In [15]:
# 1/2: 57/43
# 2/3 : 6/4
# 1/3: 65/35
p = 0
p2 = 0.725
# train_k_fold_combine(X_lgb,X_test_lgb,X_gp,X_test_gp, X_extra, X_test_extra, y_processed , w1 = p2 *(1-p), w2 = (1-p)*(1-p2), w3=p, fold_num=10)

In [16]:
y_test_pred = train_k_fold_combine_predict(X_lgb,X_test_lgb,X_gp,X_test_gp, X_extra, X_test_extra, y_processed , w1 = p2 *(1-p), w2 = (1-p)*(1-p2), w3=p, fold_num=10)

/Users/zdh/anaconda3/envs/aml_project/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/Users/zdh/anaconda3/envs/aml_project/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8


/Users/zdh/anaconda3/envs/aml_project/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/Users/zdh/anaconda3/envs/aml_project/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8


/Users/zdh/anaconda3/envs/aml_project/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/Users/zdh/anaconda3/envs/aml_project/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


...

[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8


/Users/zdh/anaconda3/envs/aml_project/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/Users/zdh/anaconda3/envs/aml_project/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8


/Users/zdh/anaconda3/envs/aml_project/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/Users/zdh/anaconda3/envs/aml_project/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8


In [17]:
final_res = np.vstack((indices_test, y_test_pred)).T
pd.DataFrame(final_res).to_csv("our_result.csv", header = ["id", "y"], index=False)